# Planet neo4j

This notebook walks through integrating "the strange planet"  custom code with Neo4j graph database. It is including `py2neo`, `ipython-cypher`, `pandas` packages.
Neo4j is graph database (it is prerequisite to be installed and running on the same machine).
Neo4j authentication should be set to require=false (otherwise causes cypher query errors).

# py2neo

`py2neo` is one of Neo4j's Python drivers. It offers a fully-featured interface for interacting with your data in Neo4j. Install `py2neo` with `pip install py2neo` or through anaconda (`conda install py2neo`) or winpython console.

### Code

Corrected the parrent-child state transition logic for cases where groups (m) >3 (replaced If constructs with For loop).

In [ ]:
# strange planet program
# rule: 2 species contribute by 1 each to produce qty 2 of the next specie (in m>3 this should allow for all comb)
# v15a - inc to v14b -jgraph +neo4j. Neo (user: neo4j, pass: test)
# initial combinations set produced based on: https://docs.python.org/2/library/itertools.html 
# logical graph implemented based on: http://interactivepython.org/runestone/static/pythonds/Graphs/Implementation.html
# Neo4j and python integration based on: http://nicolewhite.github.io/neo4j-jupyter/hello-world.html
# ...and bunch of neo4j literature (still don't get it completely, sometimes it behaves unexpected)
from itertools import combinations
from itertools import combinations_with_replacement
import numpy as np
import pandas as pd
import copy
from py2neo import authenticate, Graph
from py2neo import Node, Relationship, NodeSelector
from py2neo import Graph as PGraph
import jgraph
from neo4jrestclient.client import GraphDatabase

# used by simple logical graph construct
class Vertex:
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}

    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight

    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])

    def getConnections(self):
        return self.connectedTo.keys()

    def getId(self):
        return self.id

    def getWeight(self,nbr):
        return self.connectedTo[nbr]

# simple logical graph construct, gets job done    
class GraphI:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self,n):
        return n in self.vertList

    def addEdge(self,f,t,cost=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], cost)

    def getVertices(self):
        return self.vertList.keys()

    def __iter__(self):
        return iter(self.vertList.values())

def add_to_final_struct(g, out, out1):
    out1p = [int(e) for e in out1]
    outp = [int(e) for e in out]
    parent_node =''.join(str(e) for e in outp) # if n has more than single digit replace '' with '-' on this and next line
    child_node = ''.join(str(e) for e in (sorted((out1p), reverse=True)))    
##    print ("Parent node: ", parent_node)
##    print ("Child node: ", child_node)
    if (g.getVertex(parent_node) == None):
        g.addVertex(parent_node)
        
    parnt = g.getVertex(parent_node)
    if (g.getVertex(child_node) == None):
        g.addVertex(child_node)
    chld = g.getVertex(child_node)
    # this is for neo, need to find out what makes sense to keep parent and child the same node
#    parnt = Node("Parent", name=parent_node)
    reps = 0
    for parnt in g:
        if chld in parnt.getConnections():
            reps = parnt.getWeight(chld) # getting number of repeated trips or connections from parent to child
    # need to make increment instead of 1 in the line below
    g.addEdge(parent_node, child_node, reps+1)  # add relationship to external db table from here
##    for v in g:
##        for w in v.getConnections():
##            print("( %s, %s )" % (v.getId(), w.getId()))
    return 0

#############################################################################
## **** MAIN ****
#############################################################################

n = int(input ("Enter the total number of individuals: ")) # number of people
m = int(input("Enter the number of species: ")) # number of groups

lst = range(n, -1, -1)
alist = []
# need to replace the following 5 lines with the iterative construct as further below
# complexity of python comb_with_repl is O(r * (n! / (r! (n - r)!))), discussed at the link below
# see: http://stackoverflow.com/questions/20764926/combinations-without-using-itertools-combinations
for comb in combinations_with_replacement(lst,m):
    if sum(comb) == n:
        alist.append(comb)        
print (alist)
total_combinations = len(alist)
print ("Number of combinations: ", total_combinations)

# need comb# X comb#+1 - one more column for the row case state
# intersaction of rows and columns gives relationship for each state (row is child)
final_struct = pd.DataFrame(index = range(len(alist)-len(alist) +1), columns = [])

g = GraphI() # logical attempt, v9

# set up authentication parameters
authenticate("localhost:7474", "neo4j", "test")

# connect to authenticated graph database, v10 of program
graph = Graph("http://localhost:7474/db/data/")
db = GraphDatabase("http://localhost:7474/db/data/") # this is using REST Client
graph.delete_all()

row_cnt = 0

for comb in alist:
    count = 0
    # next For and If constructs take care of MUST_FAIL case where there is less than 2 species on the planet
    for i in range(m):
        if (comb[i] == 0):
            count = count + 1
    if (count >= m-1):
        print ("Must fail: ", comb) # need to add "must_fail" combination to db table from here
        must_fail = [int(e) for e in comb]
        p_must_fail =''.join(str(e) for e in must_fail) # for more than single digit n, change '' to '-' as separator
        if (g.getVertex(p_must_fail) == None):
            g.addVertex(p_must_fail)
        # add comb to final_struct
        temp = ','.join(map(str, comb))
#        print ("temp: " , temp)
        final_struct[0, row_cnt] = temp # am I using this???
        row_cnt = row_cnt + 1
        
    else: # there are at least 2 elements !=0, so they can produce child
        out = np.zeros(m)
        for i in range (m):
            if (comb[i] != 0):
                out[i] = comb[i]
        print ("parent: ", out)
        temp = ','.join(map(str, comb))
        final_struct[0, row_cnt] = temp
        row_cnt = row_cnt + 1

        # decrement value of j-th element (first working value in the tuple) - line 63
        for i in range(m):
            if (out[i] != 0):
                #out1 = np.zeros(m)
                out1 = copy.deepcopy(out)
                out1[i] = out[i] - 1
                for j in range(i+1, m):
                    if (out[j] != 0): #line 65 pseudocode
                        out1[j] = out[j] - 1
##                        if (j == m-1):
##                            if (j-i > 1):
##                                out1[i+1] = out[i+1] + 2
##                            else:
##                                out1[0] = out[0] + 2
##                        else:
##                            out1[j+1] = out[j+1] + 2 # line 79 pseudocode
                        out2 = copy.deepcopy(out1)
                        for k in range(m):
                            if (i != k and j != k):
                                out1[k] = out[k] + 2
                                print ("\tchild: ", out1)
                                add_to_final_struct(g, out, out1) # call to subrotine, check complexity
                                out1 = copy.deepcopy(out2)
                    else:
                        break
                    #out1 = sorted(out1, reverse=True) # need to sort outside of the loop, in add_to_final_struct
#                    print ("\tbottom: ", out1)
                    out1 = copy.deepcopy(out)
                    if (j != m-1):
                        out1[i] = out[i] - 1
                    #add_to_final_struct(final_struct, out, out1) #tbd 
                   
#print (final_struct.head) # planned to use this for adjacency matrix, by appending to df
#print ("0,1: ", final_struct.values[0,1])  # for test only
#final_mtx = final_struct.as_matrix(columns=None)
#print (final_mtx.ndim)

##neo = pd.Series(data, index = 10)
next_v = 0
next_w = 1
vertice = np.zeros(3)
node = db.labels.create("Case")

from scripts.vis import draw
options = {"State": "name"}

#v_arr = np.array(range(3), dtype=str)
v_arr = []
w_arr = np.array(range(3), dtype=str)
print (sorted(g.getVertices(), reverse=True)) # printing all vertices

query = """
MATCH (p:State)
WHERE p.name = {name}
RETURN p
"""

relship = """
START n=node(*), m=node(*)  
where exists(n.name) and exists(m.name) 
and n.name = {prev_state} 
and m.name = {next_state} 
create (n)-[:MOVES_TO]->(m)
"""

#data = graph.run(query, name="Nicole")

for v in g:
    v_curr = str(v.getId())
    x_curr = Node("State", name = v_curr) # jupyter try
    graph.create(x_curr) # jupyter try  
    
for v in g:
    v_curr = str(v.getId())
    x_curr = graph.run(query, name = v_curr) # jupyter try
    for w in v.getConnections():
        print("( %s, %s , %s )" % (v.getId(), w.getId(), v.getWeight(w))) # printing all relationships
        w_curr = str(w.getId())
        y_curr = graph.run(query, name = w_curr) # jupyter try
        graph.run(relship, prev_state = v_curr, next_state = w_curr)
          
draw(graph, options)



# ipython-cypher

`ipython-cypher` exposes `%cypher` magic in Jupyter. Install `ipython-cypher` with `pip install ipython-cypher`.

In [ ]:
%load_ext cypher

## Use parametarized Cypher to find must_fail nodes

Pass parameters to Cypher queries by passing additional key-value arguments to `Graph.cypher.execute.` Parameters in Cypher are named and are wrapped in curly braces.
Must_fail nodes are all nodes that cannot move to another state.
The cypher query is using directional directive and checks existance of possible relationships in given direction.

## Cypher to find MUST_FAIL States

Assumption is that the MUST_FAIL State definition is that it's the State that cannot produce the next State. 
To retrieve Cypher query results we use `Graph.cypher.execute` construct.

In [ ]:
results = %cypher match (p) where not (p)-[:MOVES_TO]->() return p.name as State, count(*) as Must_Fail

df = results.get_dataframe()
must_fail_State = (str(df.iloc[0][0])).strip() # setting var for later use, could also setup an array by dumping df attr

df

## DB Query for MUST_FAIL State

This is direct Neo4j DB query, it should be run on Neo4j server.
Using example from previous section it can be embedded to code to produce output in jupyter notebook.

Assumption is that the MUST_FAIL State definition is that it's the State that cannot produce the next State

In [ ]:
#1
MATCH (p) where not (p)-[:MOVES_TO]->() RETURN p.name as State, count(*) as Must_Fail
# check if the State does not go to the next State (MUST_FAIL)

## DB Query for CAN_FAIL State

This is direct Neo4j DB query, it should be run on Neo4j server. Using example from previous section it can be embedded to code to produce output in jupyter notebook.

Assumption is that the CAN_FAIL State definition is that it's any State that eventually leads to the MUST_FAIL State.

In [ ]:
#2
MATCH (e) where not (e)-[:MOVES_TO]->() with e as n MATCH (p),(n), x = shortestPath((p)-[*..15]->(n))  return p.name
# check if the State eventually goes to the MUST_FAIL State (CAN_FAIL)

## DB Query for finding non-deterministic States

This is direct Neo4j DB query, it should be run on Neo4j server. Using example from previous section it can be embedded to code to produce output in jupyter notebook.

Assumption is that the NonDeterministic State definition is that it's any State that has outgoing relationship with more than one State (including itself).

In [ ]:
#3
start n=node(*) match (n)-[r]->() return n, count(r) as rel_count order by rel_count desc
# get all nodes and the number of their outgoing relationships in descending order (NonDeterministic if count>1)